In [ ]:
pip install neo4j langchain_openai langchain_neo4j langchain_community

In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import OpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.prompts import PromptTemplate

# -----------------------------
# 1. Neo4j Connection
# -----------------------------
URI = "neo4j uri"
AUTH = ("neo4j", "your neo4j password")

graph = Neo4jGraph(
    url=URI,
    username=AUTH[0],
    password=AUTH[1]
)

# -----------------------------
# 2. Auto-load schema (ontology-lite)
# -----------------------------
print("📌 Graph Schema from Neo4j:")
print(graph.schema)   # This shows nodes, rels, properties

# -----------------------------
# 3. Custom Ontology Descriptions
# -----------------------------
ontology_hint = """
Ontology:
- A User can FOLLOW another User using :FOLLOWS.
- A User can WRITE a Post using :POSTED.
- A Person can Comment on a Post using :COMMENTED.
"""

# -----------------------------
# 4. Custom QA Prompt
# -----------------------------
qa_prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
You are an expert Cypher translator.
Here are extra ontology hints:
""" + ontology_hint + """

Given the question: {question}
Generate a Cypher query that matches the ontology and return results.
"""
)

# -----------------------------
# 5. LLM Setup
# -----------------------------
llm = OpenAI(openai_api_key="your api key")

# -----------------------------
# 6. Chain Creation
# -----------------------------
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    qa_prompt=qa_prompt,
    allow_dangerous_requests=True
)

# -----------------------------
# 7. Run Query
# -----------------------------
question = "How many followers Bob Has?"
answer = chain.run(question)

print("\n✅ Answer:", answer)
